In [2]:
import psycopg2
from psycopg2 import DatabaseError
import logging
import requests
import datetime as dt
from dateutil.relativedelta import relativedelta
# from parser_opennem import parse_request_json
# from db_populate import insert_batch
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
db_name = "energy"
db_user = "dawki"
db_pword_file = "C:\\Users\\dawki\\OneDrive\\Archive\\2019\\Documents\\private.txt"
with open(db_pword_file) as f:
    db_pword = f.read()

In [4]:
## Connect to DB
dbc = psycopg2.connect(user=db_user,
                        password=db_pword,
                        host="127.0.0.1",
                        port="5432",
                        database=db_name)
cursor = dbc.cursor()

In [5]:
# Rquest all NSW power data
cursor.execute(
    f"select interval_start, dim_region.code as region, dim_fueltech.long_name as fueltech, power \
    from fact_power, dim_batch, dim_fueltech, dim_region \
    where dim_batch.id=batch_id AND fueltech_id=dim_fueltech.id AND dim_region.id = region_id AND dim_region.code = 'NSW1'"
)
nsw_power = cursor.fetchall()

# Window data

In [6]:
nsw_power


[(datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'gas_ocgt', 0.0),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'solar_utility', 0.293),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'distillate', -0.02039),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'gas_ccgt', 0.1438),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'hydro', 71.21),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'coal_black', 7366.0),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'wind', 158.5),
 (datetime.datetime(2019, 1, 1, 0, 0), 'NSW1', 'pumps', 0.0),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'gas_ocgt', 0.0),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'solar_utility', 0.293),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'distillate', -0.02039),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'gas_ccgt', 0.1639),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'hydro', 70.91),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'coal_black', 7308.0),
 (datetime.datetime(2019, 1, 1, 0, 5), 'NSW1', 'wind'

In [14]:
nsw_power_df = pd.DataFrame(nsw_power, columns=["interval_start","region","fueltech","power"])
nsw_aggregate_power_df = nsw_power_df.groupby(["interval_start"]).sum()

# rolling average
nsw_aggregate_power_df["rolling_avg_power_month"] = nsw_aggregate_power_df["power"].rolling(window=48*31).mean()
nsw_aggregate_power_df["rolling_avg_power_week"] = nsw_aggregate_power_df["power"].rolling(window=48*7).mean()
nsw_aggregate_power_df["rolling_avg_power_day"] = nsw_aggregate_power_df["power"].rolling(window=48).mean()